## Ventilation Rate Calculation (DS/EN 16798-1 – Method 1)
This notebook implements Method 1 of the European standard DS/EN 16798-1. 
The purpose of this method is to calculate the required ventilation rate for indoor spaces 
based on the following factors:

- Indoor Environmental Quality (IEQ) category, ranging from I to IV.
- Pollution class, which represents the expected emissions in the space 
  (LPB-1 = very low, LPB-2 = low, LPB-3 = normal).
- Occupancy of the space (number of people).
- Floor area of the space in square meters.

This notebook can be used to interactively calculate ventilation requirements, 
or as a template for automated calculations across multiple rooms or buildings.



### Ventilation Tables (qp and qB)
In Method 1, two sets of values define the ventilation needs:

1. qp: required airflow per person in L/s per person, depending on the IEQ category.
2. qB: required airflow per square meter in L/s per m², depending on both IEQ category 
      and the pollution class.

These tables are taken directly from DS/EN 16798-1 standards and are used to compute 
the total ventilation rate for any space using the formula:

    Q_tot = n * qp + A * qB

where n is the number of occupants and A is the area of the space.


In [2]:
# DS/EN 16798-1 Method 1 tables (qp and qB)

QP_TABLE = {  # l/s per person
    "I": 10.0,
    "II": 7.0,
    "III": 4.0,
    "IV": 2.5,
}

QB_TABLE = {  # l/s per m2, by IEQ category and pollution class
    "I": {"LPB-1": 0.5,  "LPB-2": 1.0, "LPB-3": 2.0},
    "II": {"LPB-1": 0.35, "LPB-2": 0.7, "LPB-3": 1.4},
    "III": {"LPB-1": 0.2,  "LPB-2": 0.4, "LPB-3": 0.8},
    "IV": {"LPB-1": 0.15, "LPB-2": 0.3, "LPB-3": 0.6},
}

### Choose_ieq_category()
This function asks the user to select an Indoor Environmental Quality (IEQ) category.
The valid choices are I, II, III, or IV, which correspond to different levels of 
indoor comfort and environmental control.

The function ensures valid input by looping until the user provides a correct value.
It is used later to look up the correct airflow per person (qp) from the tables.


In [ ]:
def choose_ieq_category() -> str:
    """
    Ask user for IEQ category (I, II, III, IV).
    """
    ieq = input("Select IEQ Category (I, II, III, IV): ").strip().upper()
    valid = {"I", "II", "III", "IV"}
    while ieq not in valid:
        print("Invalid category. Please select from I, II, III, IV.")
        ieq = input("Select IEQ Category (I, II, III, IV): ").strip().upper()
    return ieq

### User Input for pollution class
This function prompts the user to select a pollution class for the space. 
Pollution classes represent the likely level of indoor pollutants, based on expected emissions 
from furniture, finishes, or occupant activities:

- LPB-1: very low polluting
- LPB-2: low polluting
- LPB-3: non low-polluting 

The input is validated in a loop to ensure correct selection. The pollution class is then
used to select the correct qB value from the ventilation tables.


In [ ]:
def choose_pollution_class() -> str:
    """
    Ask user for pollution class (LPB-1, LPB-2, LPB-3).
    """
    pollution = input(
        "Select Pollution Class "
        "(LPB-1 (very low polluting), LPB-2 (Low Polluting), LPB-3 (Non low-polluting)): "
    ).strip().upper()

    valid = {"LPB-1", "LPB-2", "LPB-3"}
    while pollution not in valid:
        print("Invalid class. Please select from LPB-1, LPB-2, LPB-3.")
        pollution = input(
            "Select Pollution Class "
            "(LPB-1 (very low polluting), LPB-2 (Low Polluting), LPB-3 (Non low-polluting)): "
        ).strip().upper()

    return pollution

### Ventilation_rate_method_1()
This function calculates the total ventilation rate (Q_tot) for a space based on:

- Number of occupants (n)
- Floor area of the space (A)
- IEQ category (to select qp)
- Pollution class (to select qB)

It also enforces a minimum requirement of 4 L/s per person, as required by the standard.
This ensures that even small rooms or low occupancy cases meet basic ventilation needs.


In [ ]:
def ventilation_rate_method_1(occupants: int, area: float,
                              ieq_category: str, pollution_class: str) -> float:
    """
    DS/EN 16798-1 Method 1:
    q_tot = n * qp + A * qB  [L/s]
    Also enforces minimum 4 L/s per person.
    """
    ieq = ieq_category.upper()
    lpb = pollution_class.upper()

    # Look up qp and qB
    qp = QP_TABLE[ieq]
    qB = QB_TABLE[ieq][lpb]

    # Base flow (people + building)
    q_tot = occupants * qp + area * qB

    # Minimum per person flow: 4 L/s per person
    q_min_people = occupants * 4.0
    q_final = max(q_tot, q_min_people)

    return q_final

### Interactive Test
This block demonstrates how the functions can be used interactively.
- It first asks the user to choose the IEQ category and pollution class.
- Then, it calculates the ventilation rate for a small example space (5 occupants, 15.2 m²).
- Finally, it prints the result formatted to two decimal places.

This example is useful for manual testing or demonstration in a classroom setting.



In [ ]:
if __name__ == "__main__":
    ieq = choose_ieq_category()
    lpb = choose_pollution_class()

    # Example: 5 people, 15.2 m2
    q = ventilation_rate_method_1(occupants=5, area=15.2,
                                  ieq_category=ieq, pollution_class=lpb)

    print(f"Calculated Ventilation Rate (Qtot): {q:.2f} L/s")
